# Example of a 3D cantilever beam modelled with brick elements
First, the necessary modules need to be imported

In [91]:
import HierAMuS
import gmsh

In a next step, gmsh needs to be initialized.

In [92]:
gmsh.initialize()
gmsh.model.add("test")

Parametrization of the geometry with
- L: length of the beam, x-direction
- h: height of the beam, z-direction
- b: width of the beam, y-direction

After the parameter definition, the 8 necessary points can be added.

In [93]:
L=10
h=1
b=1

gmsh.model.occ.addPoint(0,0,0)
gmsh.model.occ.addPoint(L,0,0)
gmsh.model.occ.addPoint(L,b,0)
gmsh.model.occ.addPoint(0,b,0)
gmsh.model.occ.addPoint(0,0,h)
gmsh.model.occ.addPoint(L,0,h)
gmsh.model.occ.addPoint(L,b,h)
gmsh.model.occ.addPoint(0,b,h)



8

After the points are defined, the 12 lines can be defined to model the 6 surfaces.

In [94]:
# Lines of lower face 1
gmsh.model.occ.addLine(1,2)
gmsh.model.occ.addLine(2,3)
gmsh.model.occ.addLine(3,4)
gmsh.model.occ.addLine(4,1)

lowerFaceLines = [1,4,3,2]

# left face liness 2
gmsh.model.occ.addLine(1,5)
gmsh.model.occ.addLine(5,8)
gmsh.model.occ.addLine(8,4)

leftFaceLines = [5,6,7,4]

# right face lines 3
gmsh.model.occ.addLine(2,6)
gmsh.model.occ.addLine(6,7)
gmsh.model.occ.addLine(7,3)

rightFaceLines = [2,10,9,8]

# top face lines 4
gmsh.model.occ.addLine(5,6)
gmsh.model.occ.addLine(7,8)

topFaceLines = [12,6,11,9]

# front face lines 5
frontFaceLines = [1,8,11,5]

# back face lines 6
backFaceLines = [3,10,12,7]

xlines = [1,3,11,12]
ylines = [2,9,4,6]
zlines = [5,7,8,10]


With the lines added, and collected counter-clockwise in the face lists, the curve loops and finally the face of the volume can be created.

When adding curveLoops, the numbering does not seem to be ordered. Therefore, the temporary variable cl is used.

In [95]:
# lower face 1
cl=gmsh.model.occ.addCurveLoop(lowerFaceLines)
f1=gmsh.model.occ.addPlaneSurface([cl])

# left face 2
cl=gmsh.model.occ.addCurveLoop(leftFaceLines)
bounFace=gmsh.model.occ.addPlaneSurface([cl])


# right face 3
cl=gmsh.model.occ.addCurveLoop(rightFaceLines)
loadFace=gmsh.model.occ.addPlaneSurface([cl])
# top face 4
cl=gmsh.model.occ.addCurveLoop(topFaceLines)
gmsh.model.occ.addPlaneSurface([cl])
# front face 5
cl=gmsh.model.occ.addCurveLoop(frontFaceLines)
gmsh.model.occ.addPlaneSurface([cl])
# top face 6
cl=gmsh.model.occ.addCurveLoop(backFaceLines)
gmsh.model.occ.addPlaneSurface([cl])



6

With the definition of the face, the definition of the volume can be added. Then the geometry is completed and must be synchronized in gmsh.

In [96]:
gmsh.model.occ.addSurfaceLoop([1,2,3,4,5,6])
gmsh.model.occ.addVolume([1])


gmsh.model.occ.synchronize()



Now, the mesh can be prepared for meshing, by setting transfinite lines, face and volumes. Furthermore, the parameters for number of nodes per direction are introduces as:
- nx: number of nodes in x-direction
- ny: number of nodes in y-direction
- nz: number of nodes in z-direction

In [97]:
nx=30
ny=7
nz=7

# settings the lines to be transfinite
for i in xlines:
    gmsh.model.mesh.setTransfiniteCurve(i,nx)
for i in ylines:
    gmsh.model.mesh.setTransfiniteCurve(i,ny)
for i in zlines:
    gmsh.model.mesh.setTransfiniteCurve(i,nz)
    
# setting the faces to be transfinite
for i in [1,2,3,4,5,6]:
    gmsh.model.mesh.setTransfiniteSurface(i)
    
# setting the volume to be transfinite
gmsh.model.mesh.setTransfiniteVolume(1)


gmsh.option.setNumber("Mesh.RecombineAll", 1) # This option sets gmsh to recombine tetra to bricks

#gmsh.fltk.run()
gmsh.model.mesh.generate(3)



After the mesh is generated, the finite element code can be initialized. And the commands can be requested from the FE-system.

In [98]:
fesys = HierAMuS.FEMPy("./","BrickCantilever")
fesys.setStaticSolutionState()
fesys.setSolver(2)

mesh = fesys.getMeshCommands()
macro = fesys.getMacroCommands()
gm = mesh.getFromGMESH()
geo = mesh.getGeometryCommands()

macro.setLogLevel(fesys.FullLog(),fesys.FullLog())

With the commands, the mesh can be transfered to the finite element code as geometry objects. And the remaining necessary information can be generated using the geometry command checkGeometry.

In [99]:
gm.addGeomFromGmsh(gmsh)
geo.checkGeometry()

Time to add geometry from Gmsh:  0.10903410000003078 s

Geometry check and update took:  0.05886659999998756  s



After the geometry is transfered, the finite elements must be created. This is done with the gmsh command addBrickVolumeFiniteElements from the gm object.

The parameters are:
- gmsh: the gmsh object itself
- order: the order of the gmsh elements, currently only order 1 is supported
- volumeTags: the volume tag of the gmsh geometry part (see gmsh.mmodel.occ.addVolume), we added only 1 volume, so it has number 1.
- material: the material number which will be assigned to the finite elements

In [100]:
gm.addBrickVolumeFiniteElements(gmsh=gmsh,order=1,volumeTags=1,material=1)

After the finite elements are created, and a material number is assigned to them, the specific material must be created. A material is a set of a material formulation and an element formulation.

The element formulation can be added using the object ElementFormulations, which can be acquired using the mesh commands object by mesh.getElementFormulations(). As bricks are used, we use a 3DSolid element with the command addEL300_3DSolid. Its parameters are:
- num: number of the element formulation, later used to define the material set.
- meshiddisp: the mesh id of the displacement field
- disporder: the shape function order to approximate the displacements.
- mode: the mode of the elementformulation: 1: geometrical linear, 2: geometrical nonlinear

Like the element formulation a material formulation is required. Here, we use an isotropic linear elastic material. The material formulation can be added using the mesh.getMaterialFormulations() object. It will be added by the command addMA1_3D_LinearElastic_Isotrop, with the parameters:
- number: number of the material formulation which will be used when adding the material set.
- E: Young's modulus
- nu: Poisson ratio

Finally, the material set can be added using the mesh command addMaterial. It has the parameters:
- matNum: the material set number, which is used in the addBrickVolumeFiniteElements command.
- matFormNum: the number of the materialformulation which should be used, same number as used in addMA1_3D_LinearElastic_Isotrop
- elemFormNum: the number of the elementformulation which should be used, same number as used in addEL300_3DSolid

In [101]:
mesh.getElementFormulations().addEL300_3DSolid(num=1,meshiddisp=1,disporder=1,mode=1)
mesh.getMaterialFormulations().addMA1_3D_LinearElastic_Isotrop(number=1,E=1,nu=0.3)
mesh.addMaterial(matNum=1,matFormNum=1,elemFormNum=1)

After adding all elements and the required material sets, the degrees of freedom can be set using the mesh command setDegreesOfFreedom

In [102]:
mesh.setDegreesOfFreedom()

After the degrees of freedom are distributed, the boundary conditions can be set. As we are dealing with a volume model, the boundary condition are set on a face. First, we acquire the face numbers from gmsh with the command gm.getFaceNumbers. The parameters for the command are:
- gmsh: gmsh itself
- tagin: the geometry tag from gmsh of the surface, see part where the faces are created
- ftype: the type of the face; 3: 3 corner nodes; 4: 4 corner nodes. Here we used brick elements, therefore we have quadrilateral elements, thus the ftype must be 4
- order: the order of the elements, here 1
The command returns a list of all the faces.

After retrieving the face numbers, the boundary conditions can be set with the command mesh.getBoundaryConditions().singleBC. The parameters are:
- eltype: The geometric element type on which the boundary conditions are set; Here we set it on faces. The geometric type can be retrieved from the geo commands object.
- number: The numbers of the faces on which the boundary conditions need to be set; retrieved from gmsh
- meshId: The mesh id of the nodes which should be affected by the boundary conditions.
- dofs: The dofs which should be affected; each node has exactly 3 degrees of freedom (u_x, u_y, u_z here). 1 means the degree of freedom is changed to fixed, 0: means the status of the degree of freedom is not affected if set=False, otherwise its status is set to free
- shapeOrder: Shape function order for the field associated with the given mesh id
- set: If True, the boundary conditions are set (free and fixed); if False then the boundary conditions are only added, meaning that the status of the degrees of freedom will not be set to free if it already was set to fixed.


In [103]:
fnums = gm.getFaceNumbers(gmsh=gmsh,tagin=bounFace,ftype=4,order=1)
mesh.getBoundaryConditions().singleBC(eltype=geo.faceType(),number=fnums,meshId=1,dofs=[1,1,1],shapeOrder=1,set=True)

Like the supports, the load can be applied to a face as well with kind of an comparable approach. First, the face element numbers must be retrieved from gmsh.

Loads are added using the mesh.getBoundaryConditions().singleLoad command. Parameters are the same as for singleBC, except:
- load: The load in the 3 directions, here a face load in y-direction is applied.
- propnum: associates the load with the proportional load function (here one), see definition of proportional load functions later on
- shapeorder: should be the same as specified when adding the element formulation. It is required the integrate the load correctly over the face elements.

In [104]:
fnums = gm.getFaceNumbers(gmsh=gmsh,tagin=loadFace,ftype=4,order=1)
print(loadFace)
mesh.getBoundaryConditions().singleLoad(eltype=geo.faceType(),number=fnums,meshId=1,load=[0,1,0],propnum=1,shapeorder=1)

3


When all boundary conditions are set, the sparse matrix can be setup using the macro command sparseSetUp.

In [105]:
gmsh.finalize()
macro.sparseSetUp()

Before performing the calculation, the proportional load function needs to be added. This is done using the command macro.setPropFunction. The parameters are:
- number: The number of the prop function, associated with propnum when adding loads
- function: is a python function of a single variable computing the load factor in dependence of the time t. Here, the load function is t^2
- tmin: the time value after which the function is valid
- tmax: the time function until the function is valid

Afterwards, we have to set delta t, the value to increase the time with when the timeincr command is used.
Next, the time is incremented by the command macro.timeincr

In [106]:
macro.setPropFunction(number=1,function=lambda t: t**2,tmin=0,tmax=1000)
macro.setDt(dt=1)
macro.timeincr()

After the time is incremented, the system can be solved with an appropriate macro command. Either 
macro.newton() : which performs newton iterations to solve the system

or

macro.assembleSolve() : which assembles the system and solves it one time, which is sufficient in case of a linear calculation

In [107]:
macro.assembleSolve()

After the system is solved, the solution can be written to a paraview file with the plot command fesys.getPlotCommands().toFile() .
This will create a folder parvout in the directory which was specified, when creating the finite element object with fesys = HierAMuS.FEMPy("./","BrickCantilever"). The first argument is the folder in which the parvout folder is created.

In [108]:
fesys.getPlotCommands().toFile()